In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable

## Get Last Run time

In [0]:
# Set a configuration variable for the last ingest time
last_ingest_df = spark.sql("SELECT last_timestamp FROM control.ctl.control_dates WHERE stage_name = 'silver_transformation'")
last_ingest_time = last_ingest_df.collect()[0]['last_timestamp']
spark.conf.set("last_ingest_time", str(last_ingest_time))

## Reading Data

In [0]:
%sql
USE CATALOG bronze;

In [0]:
# The 2 bronze tables we want to clean/normalise
# Compare to configuration variable to only process newly received data
df_contacted = spark.read.table("customers.contacted")\
    .filter(col("ingest_timestamp") > spark.conf.get("last_ingest_time"))\
    .drop("_rescued_data")\
    .withColumn("ingest_timestamp", date_format(col("ingest_timestamp"), "yyyy-MM-dd HH:mm:ss"))    
df_contact_details = spark.read.table("customers.contact_details")\
    .filter(col("ingest_timestamp") > spark.conf.get("last_ingest_time"))\
    .drop("_rescued_data")\
    .withColumn("ingest_timestamp", date_format(col("ingest_timestamp"), "yyyy-MM-dd HH:mm:ss"))

## Cleaning Data

Normalise into:
- customer
- customer_email
- customer_address
- contact_type_lookup
- contact_history


In [0]:
# The base customer table will be the reference for other customer details
df_customer = df_contact_details.select("customer_id", "first_name", "last_name", "ingest_timestamp")

# All the normalised tables should only have 1 reference to each customer_id
# Order this by most recently received data
customer_dedupe_window_spec = Window.partitionBy("customer_id").orderBy(desc("ingest_timestamp"))

# Base customer
df_customer = df_customer\
    .withColumn("rn", row_number().over(customer_dedupe_window_spec))\
    .filter(col("rn") == 1)\
    .drop("rn")

# Customer email only needs records for customers who have an email
df_customer_email = df_contact_details.select("customer_id", "email", "ingest_timestamp")\
    .filter(col("email").isNotNull())\
    .withColumnRenamed("email", "email_address")\
    .dropDuplicates()

# Customer address only needs records for customers who have a valid zip code
df_customer_address = df_contact_details.select("customer_id", "address", "city", "state", "zip_code", "ingest_timestamp")\
    .filter(col("zip_code") != "INVALID")\
    .dropDuplicates()

# Dedupe
df_customer_email = df_customer_email\
    .withColumn("rn", row_number().over(customer_dedupe_window_spec))\
    .filter(col("rn") == 1)\
    .drop("rn")

df_customer_details = df_customer_details\
    .withColumn("rn", row_number().over(customer_dedupe_window_spec))\
    .filter(col("rn") == 1)\
    .drop("rn")

In [0]:
# Contact type is a lookup table
df_contact_type = df_contacted.select("contact_type", "ingest_timestamp")

# Only need 1 record for each contact type
df_contact_type = df_contact_type\
    .withColumn("rn", row_number().over(Window.partitionBy("contact_type").orderBy(lit(1))))\
    .filter(col("rn") == 1)\
    .drop("rn")

### Transform contact_type_lookup

This will allow us to use the contact_type_key column in the contact_history table

In [0]:
# Create a dataframe of exising records as a reference to see if we have received new contact types
# At the same time, store a reference to whether the table exists (used for merge later)
# And find the max key (0 if the table doesn't exist) so we can create keys above that value
if spark.catalog.tableExists("silver.customers.contact_type_lookup"):
    con_lookup_exists = True
    max_con_lookup_key = spark.sql("SELECT MAX(contact_type_key) AS max_con_lookup_key FROM silver.customers.contact_type_lookup").collect()[0]['max_con_lookup_key']
    df_existing_lookup_records = spark.sql("SELECT contact_type_key, contact_type, date_created, last_updated FROM silver.customers.contact_type_lookup")
else:
    con_lookup_exists = False
    max_con_lookup_key = 0
    df_existing_lookup_records = spark.createDataFrame([], schema = StructType([StructField("contact_type_key", IntegerType()), StructField("contact_type", StringType()), StructField("date_created", TimestampType()), StructField("last_updated", TimestampType())]))

# Rename columns to avoid conflicts for join
df_existing_lookup_records = df_existing_lookup_records\
    .withColumnRenamed("contact_type_key", "existing_contact_type_key")\
    .withColumnRenamed("contact_type", "existing_contact_type")\
    .withColumnRenamed("date_created", "existing_date_created")\
    .withColumnRenamed("last_updated", "existing_last_updated")

# Left join to split old and new records
# If the left join doesn't have a match, the existing_contact_type_key will be null
# If the left join has a match, the existing_contact_type_key will be populated
df_joined_lookup = df_contact_type.join(df_existing_lookup_records, on = df_existing_lookup_records.existing_contact_type == df_contact_type.contact_type, how = "left")
df_existing_lookup_records = df_joined_lookup\
    .filter(col("existing_contact_type_key").isNotNull())
df_new_lookup_records = df_joined_lookup\
    .filter(col("existing_contact_type_key").isNull())

# Create a Window for the row_number() function
# This ensures that any newly created contact_key is unique, incremental, and repeatable in case of failure
# contact_type has already been deduped, so row_number() behaviour is predictable
con_lookup_window_spec = Window.orderBy("contact_type")

# Format new records, ingest_timestamp is both the date_created and last_updated
# Create key by using row_number() added on to the max key found earlier
# Don't need to keep null fields from the join or ingest_timestamp
df_new_lookup_records = df_new_lookup_records\
    .withColumn("contact_type_key", row_number().over(con_lookup_window_spec) + lit(max_con_lookup_key))\
    .withColumn("date_created", col("ingest_timestamp"))\
    .withColumn("last_updated", col("ingest_timestamp"))\
    .drop("existing_contact_type_key", "existing_contact_type", "existing_date_created", "existing_last_updated", "ingest_timestamp")

# Key already exists so just rename the column back to contact_type_key
# date_created should be the same as the existing date_created
# ingest_timestamp becomes last_updated
# Drop duplicate columns
df_existing_lookup_records = df_existing_lookup_records\
    .withColumn("last_updated", col("ingest_timestamp"))\
    .withColumnRenamed("existing_contact_type_key", "contact_type_key")\
    .withColumnRenamed("existing_date_created", "date_created")\
    .drop("existing_contact_type", "existing_last_updated", "ingest_timestamp")

# Union the old and new records
df_lookup_final = df_existing_lookup_records.unionByName(df_new_lookup_records)


### Get contact_type_key into contact_history

In [0]:
# Replace contact_type in our contacted dataframe with the new contact_type_key created above
df_contacted = df_contacted.join(df_lookup_final, on = df_contacted.contact_type == df_lookup_final.contact_type, how = "inner")\
    .select("contact_id", "customer_id", "campaign_id", "contact_type_key", "contact_date", "ingest_timestamp")

In [0]:
# Same logic as above
# Keep reference of if contact_history already exists for later merge
# Find max surrogate key
# Create dataframe of existing records for finding new vs updated records
if spark.catalog.tableExists("silver.customers.contact_history"):
    con_history_exists = True
    max_con_history_key = spark.sql("SELECT MAX(contact_key) AS max_con_history_key FROM silver.customers.contact_history").collect()[0]['max_con_history_key']
    df_existing_contact_history = spark.sql("SELECT contact_id, contact_key, date_created, last_updated FROM silver.customers.contact_history")
else:
    con_history_exists = False
    max_con_history_key = 0
    df_existing_contact_history = spark.createDataFrame([], schema = StructType([StructField("contact_id", StringType()), StructField("contact_key", IntegerType()), StructField("date_created", TimestampType()), StructField("last_updated", TimestampType())]))

# Rename to avoid join conflicts
df_existing_contact_history = df_existing_contact_history\
    .withColumnRenamed("contact_key", "existing_contact_key")\
    .withColumnRenamed("contact_id", "existing_contact_id")\
    .withColumnRenamed("date_created", "existing_date_created")\
    .withColumnRenamed("last_updated", "existing_last_updated")

# Split new vs updated records
df_joined_contact_history = df_contacted.join(df_existing_contact_history, on = df_existing_contact_history.existing_contact_id == df_contacted.contact_id, how = "left")
df_existing_contact_history = df_joined_contact_history\
    .filter(col("existing_contact_key").isNotNull())
df_new_contact_history = df_joined_contact_history\
    .filter(col("existing_contact_key").isNull())

con_history_window_spec = Window.orderBy("contact_id")

df_new_contact_history = df_new_contact_history\
    .withColumn("contact_key", row_number().over(con_history_window_spec) + lit(max_con_history_key))\
    .withColumn("date_created", col("ingest_timestamp"))\
    .withColumn("last_updated", col("ingest_timestamp"))\
    .drop("existing_contact_key", "existing_contact_id", "existing_date_created", "existing_last_updated", "ingest_timestamp")

df_existing_contact_history = df_existing_contact_history\
    .withColumn("last_updated", col("ingest_timestamp"))\
    .withColumnRenamed("existing_contact_key", "contact_key")\
    .withColumnRenamed("existing_date_created", "date_created")\
    .drop("existing_contact_id", "existing_last_updated", "ingest_timestamp")

df_contact_history_final = df_existing_contact_history.unionByName(df_new_contact_history)

### Transform customer

As above, this will allow us to use customer_key in customer_email and customer_address

In [0]:
# Same logic again
if spark.catalog.tableExists("silver.customers.customer"):
    cust_table_exists = True
    max_cust_key = spark.sql("SELECT MAX(customer_key) AS max_cust_key FROM silver.customers.customer").collect()[0]['max_cust_key']
    df_existing_customer = spark.sql("SELECT customer_key, customer_id, date_created, last_updated FROM silver.customers.customer")
else:
    cust_table_exists = False
    max_cust_key = 0
    df_existing_customer = spark.createDataFrame([], schema = StructType([StructField("customer_key", IntegerType()), StructField("customer_id", StringType()), StructField("date_created", TimestampType()), StructField("last_updated", TimestampType())]))

df_existing_customer = df_existing_customer\
    .withColumnRenamed("customer_key", "existing_customer_key")\
    .withColumnRenamed("customer_id", "existing_customer_id")\
    .withColumnRenamed("date_created", "existing_date_created")\
    .withColumnRenamed("last_updated", "existing_last_updated")

df_joined_customer = df_customer.join(df_existing_customer, on = df_existing_customer.existing_customer_id == df_customer.customer_id, how = "left")
df_existing_customer = df_joined_customer\
    .filter(col("existing_customer_key").isNotNull())
df_new_customer = df_joined_customer\
    .filter(col("existing_customer_key").isNull())

cust_window_spec = Window.orderBy("customer_id")

df_new_customer = df_new_customer\
    .withColumn("customer_key", row_number().over(cust_window_spec) + lit(max_cust_key))\
    .withColumn("date_created", col("ingest_timestamp"))\
    .withColumn("last_updated", col("ingest_timestamp"))\
    .drop("existing_customer_key", "existing_customer_id", "existing_date_created", "existing_last_updated", "ingest_timestamp")

df_existing_customer = df_existing_customer\
    .withColumn("last_updated", col("ingest_timestamp"))\
    .withColumnRenamed("existing_customer_key", "customer_key")\
    .withColumnRenamed("existing_date_created", "date_created")\
    .drop("existing_customer_id", "existing_last_updated", "ingest_timestamp")

df_customer_final = df_existing_customer.unionByName(df_new_customer)

### Get customer_key into customer_email and customer_address

In [0]:
# Replace customer_id with newly created customer_key in our customer_email dataframe
df_customer_email = df_customer_email.join(df_customer_final, on = df_customer_email.customer_id == df_customer_final.customer_id, how = "inner")\
    .select(df_customer_final.customer_key, df_customer_email.email_address, df_customer_email.ingest_timestamp)

In [0]:
# Do the same with our customer_address dataframe
# Also read from our state_regions lookup to add region info to the address
df_state_regions = spark.read.table("silver.customers.state_region")

df_customer_address = df_customer_address\
    .join(df_customer_final, on = df_customer_address.customer_id == df_customer_final.customer_id, how = "inner")\
    .join(df_state_regions, on = df_customer_address.state == df_state_regions.state, how = "left")\
    .select(df_customer_final.customer_key, df_customer_address.address, df_customer_address.city, df_customer_address.state, df_state_regions.region_id, df_customer_address.zip_code, df_customer_address.ingest_timestamp)\
    .withColumn("region_id", when(col("region_id").isNull(), lit("UNKNOWN")).otherwise(col("region_id")))

## Merge into silver layer

Merge into or create silver.customers.contact_type_lookup

In [0]:
if con_lookup_exists:
    dlt_con_lookup = DeltaTable.forName(spark, "silver.customers.contact_type_lookup")
    dlt_con_lookup.alias("t")\
        .merge(df_lookup_final.alias("s"), "t.contact_type_key = s.contact_type_key")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_lookup_final.write.mode("overwrite").saveAsTable("silver.customers.contact_type_lookup")

Merge into or create silver.customers.contact_history

In [0]:
if con_history_exists:
    dlt_history = DeltaTable.forName(spark, "silver.customers.contact_history")
    dlt_history.alias("t")\
        .merge(df_contact_history_final.alias("s"), "t.contact_key = s.contact_key")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_contact_history_final.write.mode("overwrite").saveAsTable("silver.customers.contact_history")

Merge into or create silver.customers.customer

In [0]:
if cust_table_exists:
    dlt_cust = DeltaTable.forName(spark, "silver.customers.customer")
    dlt_cust.alias("t")\
        .merge(df_customer_final.alias("s"), "t.customer_key = s.customer_key")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_customer_final.write.mode("overwrite").saveAsTable("silver.customers.customer")

Merge into or create silver.customers.customer_email

In [0]:
# Email is a bit different to above
# No surrogate key was created as each customer should only have 1 email on the system
# So we merge using customer_key
# We also don't have a boolean variable already to see if the table exists,
# so must use the spark.catalog
if spark.catalog.tableExists("silver.customers.customer_email"):
    dlt_cust_email = DeltaTable.forName(spark, "silver.customers.customer_email")
    dlt_cust_email.alias("t")\
        .merge(df_customer_email.alias("s"), "t.customer_key = s.customer_key")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_customer_email.write.mode("overwrite").saveAsTable("silver.customers.customer_email")

Merge into or create silver.customers.customer_address:

In [0]:
# Address is the same as email above
if spark.catalog.tableExists("silver.customers.customer_address"):
    dlt_cust_addr = DeltaTable.forName(spark, "silver.customers.customer_address")
    dlt_cust_addr.alias("t")\
        .merge(df_customer_address.alias("s"), "t.customer_key = s.customer_key")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_customer_address.write.mode("overwrite").saveAsTable("silver.customers.customer_address")